<a href="https://colab.research.google.com/github/Nwanna-Joseph/nlp_week_1_solution/blob/week3/Copy_of_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [2]:
!rm __MACOSX/._train1.txt __MACOSX/._train2.txt __MACOSX/._valid1.txt __MACOSX/._valid2.txt train1.txt train2.txt valid1.txt valid2.txt
!wget -O download.zip https://github.com/Nwanna-Joseph/nlp_week_1_solution/blob/week3/train_valid_files.zip?raw=true
!unzip download.zip
!rm download.zip

rm: cannot remove '__MACOSX/._train1.txt': No such file or directory
rm: cannot remove '__MACOSX/._train2.txt': No such file or directory
rm: cannot remove '__MACOSX/._valid1.txt': No such file or directory
rm: cannot remove '__MACOSX/._valid2.txt': No such file or directory
rm: cannot remove 'train1.txt': No such file or directory
rm: cannot remove 'train2.txt': No such file or directory
rm: cannot remove 'valid1.txt': No such file or directory
rm: cannot remove 'valid2.txt': No such file or directory
--2022-05-16 06:23:53--  https://github.com/Nwanna-Joseph/nlp_week_1_solution/blob/week3/train_valid_files.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Nwanna-Joseph/nlp_week_1_solution/raw/week3/train_valid_files.zip [following]
--2022-05-16 06:23:53--  https://github.com/Nwanna-Joseph/nlp_week_1_solution/raw/week3/train_v

In [3]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [4]:
print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [5]:
len(train_data)

5561

In [6]:
for each_sentence in [['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']]:
  buffer = []
  for position,each_word in enumerate(each_sentence):

    n=3

    if(len(buffer) >= n):
      buffer.pop(0)

    print(buffer,each_word)
    buffer.append(each_word)


[] <s>
['<s>'] my
['<s>', 'my'] fathers
['my', 'fathers'] don't
['fathers', "don't"] speak
["don't", 'speak'] dutch.
['speak', 'dutch.'] </s>


In [7]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []
    print()
    ## FILL CODE
    for each_sentence in data:
      sentence = each_sentence
      sentence_copy = []
      for each_word in sentence:
        word = each_word
        word_frequency = vocab.get(word)

        if(word_frequency == None):
          sentence_copy.append('<unk>')
          continue
        
        if ( word_frequency < mincount):
          sentence_copy.append('<unk>')
          continue

        sentence_copy.append(word)

      data_with_unk.append(sentence_copy)

    return data_with_unk

In [8]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


In [9]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = {}
    word_frequency = {}

    for sentence in data:
        sentence = tuple(sentence)
        
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value

        # we'll be using stupid back off

        for position,each_word in enumerate(sentence):
          word = each_word
          word_tuple = tuple([word])
          
          last_n_words = []
          for i in range(n-1): #index from current word to last n chars, n_gram
            
            last_i_word = position-i-1
            if(last_i_word < 0):
              continue

            last_n_words.insert(0,sentence[last_i_word])

            #convert list to tuple and index
            last_n_words_tuple = tuple(last_n_words)

            if(counts.get(last_n_words_tuple) == None):
              counts[last_n_words_tuple] = {}
            
            word_count = counts.get(last_n_words_tuple,{}).get(word_tuple,0)
            
            counts[last_n_words_tuple][word_tuple] = word_count + 1

            #update frequency lookup
            word_frequency[last_n_words_tuple] = word_frequency.get(last_n_words_tuple,0) +1

    # print(word_frequency)
    # print(word_frequency[tuple(['how'])])
    # print(counts[tuple(['<s>'])])
    # print(counts)
    # return

    prob = {}
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL CODE
      context_total_appearance = word_frequency[context]

      # if counts[context]
      
      for entry in counts[context]:
        if (prob.get(context) == None):
          prob[context] = {}
        prob[context][entry] = counts[context][entry] / context_total_appearance

    # print(prob)

    return prob

In [10]:
['a','b','c','d','e'][2-1:2-2:-1]

['b']

In [11]:
tuple([1])

(1,)

In [12]:
# RUN TO BUILD NGRAM MODEL

n = 3
print("build ngram model with n = ", n)
# print(train_data[:2])
model = build_ngram(train_data[:], n)

build ngram model with n =  3


In [ ]:
model

Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [14]:
[val for val in vocab.values()]

[5561,
 203,
 1,
 199,
 12,
 1,
 5561,
 4,
 735,
 4669,
 3,
 430,
 5,
 16,
 1761,
 44,
 246,
 6,
 2,
 258,
 12,
 1478,
 104,
 26,
 900,
 21,
 24,
 23,
 980,
 1,
 72,
 36,
 1,
 8,
 1,
 1,
 1,
 222,
 507,
 10,
 25,
 1,
 70,
 7,
 62,
 11,
 329,
 356,
 22,
 1103,
 788,
 283,
 835,
 192,
 15,
 6,
 14,
 582,
 274,
 6,
 351,
 3,
 127,
 1045,
 35,
 3,
 2,
 17,
 6,
 36,
 107,
 166,
 75,
 16,
 1,
 59,
 266,
 27,
 758,
 166,
 45,
 6,
 18,
 1,
 1,
 45,
 2,
 155,
 9,
 6,
 1,
 19,
 80,
 692,
 3,
 77,
 3,
 2,
 47,
 2,
 2,
 10,
 299,
 2,
 190,
 167,
 5,
 119,
 195,
 54,
 110,
 10,
 102,
 6,
 33,
 14,
 220,
 1,
 1,
 24,
 45,
 15,
 6,
 174,
 2,
 1,
 47,
 145,
 48,
 19,
 22,
 3,
 3,
 129,
 142,
 53,
 33,
 76,
 6,
 18,
 38,
 8,
 5,
 2,
 77,
 4,
 112,
 3,
 58,
 7,
 3,
 5,
 2,
 40,
 4,
 1,
 220,
 1,
 1,
 4,
 3,
 1,
 1,
 63,
 3,
 3,
 1,
 226,
 2,
 4,
 44,
 2,
 2,
 1,
 53,
 464,
 211,
 217,
 5,
 6,
 13,
 7,
 2,
 1,
 7,
 8,
 2,
 114,
 1,
 43,
 51,
 112,
 28,
 3,
 2,
 8,
 44,
 126,
 1,
 6,
 1,
 5,
 1,
 4,
 16,


In [15]:
total_words = sum([val for val in vocab.values()])
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE

    length = len(context)
    backoff = 1
    word_tuple = tuple([w])
    for i in range(len(context)):
      i_to_tuple = tuple(context[i:])
      # print(i_to_tuple,"|",word_tuple)

      prob = model.get(i_to_tuple,{}).get(word_tuple)
       
      if prob == None:
        backoff * 0.4
      else:
        return prob * backoff
        
    return backoff * (vocab.get(w,0)/total_words)

In [16]:
get_prob(model, ['<s>', 'my', 'fathers', "don't", 'speak'], 'dutch.')

0.3333333333333333

In [17]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''

    ## FILL CODE
    prob = 1
    for each_sentence in data:
      buffer = []
      for position,each_word in enumerate(each_sentence):
        if(len(buffer) >= n):
          buffer.pop(0)
        prob += get_prob(model, buffer, each_word)
        buffer.append(each_word)

    return prob

In [18]:
valid_data

[['<s>',
  'we',
  '<unk>',
  'over',
  'the',
  'decision',
  'to',
  'do',
  'that.',
  '</s>'],
 ['<s>', 'sami', 'should', 'be', 'aware', 'of', 'the', 'situation.', '</s>'],
 ['<s>', 'my', 'father', "won't", 'allow', 'that.', '</s>'],
 ['<s>',
  'tom',
  'goes',
  'to',
  'the',
  'gym',
  'three',
  'times',
  'a',
  'week.',
  '</s>'],
 ['<s>', 'ich', '<unk>', '<unk>', '<unk>', '</s>'],
 ['<s>',
  'his',
  'speech',
  'was',
  'an',
  'effective',
  '<unk>',
  'for',
  'the',
  "government's",
  '<unk>',
  '</s>'],
 ['<s>',
  "weren't",
  'you',
  'the',
  'one',
  'who',
  'was',
  'supposed',
  'to',
  'tell',
  'tom',
  'and',
  'mary',
  'they',
  "wouldn't",
  'be',
  'allowed',
  'to',
  'do',
  'that?',
  '</s>'],
 ['<s>',
  '"why',
  'do',
  'you',
  'rob',
  '<unk>',
  '<unk>',
  'that’s',
  'where',
  'the',
  'money',
  '<unk>',
  '</s>'],
 ['<s>', 'are', 'these', 'children', 'yours?', '</s>'],
 ['<s>', 'this', 'park', "isn't", 'open', 'to', 'the', '<unk>', '</s>'],
 ['

In [19]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 138.72070613113132


In [20]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE

    for _ in range(len(context)):
      #check
      map = model[tuple([context])]
      if map == None:
        #no match, remove pop(0)
        context.pop(0)
        #retry
        continue
      
      #match, return nexts...
      return map

In [21]:
['1','2','3'][-1]

'3'

In [22]:
rr = ['1']
for _ in range(len(rr)):
  print(rr)
  rr.pop(0)
print(rr)

['1']
[]


In [56]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        context = sentence[-1]
        next_choice_of_words = get_proba_distrib(model,context)
        next_choice_of_words_lists = [k[0] for k in next_choice_of_words.keys()]
        # print(next_choice_of_words_lists)
        # print(next_choice_of_words)
        choice = np.random.choice(next_choice_of_words_lists, 1, replace=False)
        sentence.append(choice[0])
        
    return sentence

In [57]:
# GENERATE A SENTENCE FROM THE MODEL


print("Generated sentence: ",generate(model))

<s>
lady
was
haunted.
Generated sentence:  ['<s>', 'lady', 'was', 'haunted.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.